# Day 4

[Day 4 description](https://adventofcode.com/2018/day/4)

I think that the main problem with this one was to understand what was going on. After figuring it out, the solution is pretty straighforward. I'll make use of `datetime` module to simplify the parsing and the `Counter` as usual.

In [1]:
import re
from datetime import datetime, timedelta
from collections import Counter, defaultdict

In [2]:
rows = []
with open('AOC2018_04_input.txt') as f:
    for raw in f.readlines():
        rows.append(raw.strip())
# due to datetime format, we can sort rows lexicographically
rows.sort()

We don't know a priori how many actions will take every guard. We need to group logs by id, then compute how much time is a guard asleep during his turn.

In [3]:
def parse_log(row):
    date = datetime.strptime(row[1:17], "%Y-%m-%d %H:%M")
    action, *rest = row[19:].split(" ")
    if action == 'Guard':
        guard_number = rest[0]
        awaken = True
    else:
        guard_number = None
        awaken = (action == 'wakes')
    return [date, guard_number, awaken]

In [4]:
parsed_rows = [parse_log(row) for row in rows]

In [5]:
last_id = None
log_by_id = defaultdict(list)
for row in rows:
    d, guard_id, a = parse_log(row)
    if guard_id is None:
        guard_id = last_id
    else:
        last_id = guard_id
    log_by_id[last_id].append((d, a))

**Strategy 1**: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

i.e., rank guards by total minutes asleep, then select the single most frequent minute (regardless of the hour).
In order to solve this (and the next part too), for each guard id we collect a counter that keeps track of how many times that guard was asleep during that minute.

In [6]:
def get_minutes_asleep(events):
    minutes_when_asleep = Counter()
    for event_start, event_stop in zip(events[:-1], events[1:]):
        time_start, asleep_start = event_start
        time_end, asleep_end  = event_stop
        if not asleep_start and asleep_end:
            # guard is asleep now!
            minutes_asleep = (time_end - time_start).seconds // 60
            minutes_when_asleep.update([(time_start + timedelta(minutes=i)).minute for i in range(minutes_asleep)])
    return minutes_when_asleep

In [7]:
minutes_by_id = dict()
for id_, events in log_by_id.items():
    mas = get_minutes_asleep(events)
    minutes_by_id[id_] = mas

In [8]:
# guard with most total minutes asleep
guard_id = max([(k, sum(v.values())) for k, v in minutes_by_id.items()], key=lambda x: x[1])[0]

# most frequent minute for that guard
best_minute = minutes_by_id[guard_id].most_common(1)[0][0]

int(guard_id[1:]) * best_minute

84636

**Strategy 2**: Of all guards, which guard is most frequently asleep on the same minute?

i.e. for each guard, rank minutes by times asleep in that minute; then select guard with best record.

In [9]:
# select best minute by guard id
record_by_id = dict()
for guard_id, minutes in minutes_by_id.items():
    best_minute = minutes.most_common(1)
    if len(best_minute) == 1:
        record_by_id[guard_id] = best_minute[0]

In [10]:
guard_id, (best_minute, _) = max(record_by_id.items(), key=lambda x: x[1][1])

int(guard_id[1:]) * best_minute

91679